Query splitter

In [1]:
import pandas as pd
from collections import defaultdict
import os
import csv

In [2]:
def delete_first_row_tsv(file_path):
    temp_file_path = file_path + '.temp'

    with open(file_path, 'r', newline='') as f_input, open(temp_file_path, 'w', newline='') as f_output:
        reader = csv.reader(f_input, delimiter='\t')
        writer = csv.writer(f_output, delimiter='\t')

        next(reader)

        for row in reader:
            writer.writerow(row)

    os.rename(temp_file_path, file_path)

In [3]:
qrels_df = pd.read_csv('/home/requirements/data/MSMARCO/qrels.dev.tsv', names=['qid', '0', 'pid', '1'], sep='\t')
quries_df = pd.read_csv('/home/requirements/data/MSMARCO/queries.dev.tsv', names=['qid', 'text'], sep='\t')
grouped_qrels_df = qrels_df.groupby('qid')

In [20]:
qrels = defaultdict(list)
quries = defaultdict(list)

for i, group in grouped_qrels_df:
    idx = len(group) - 1
    for _, (qid, _, pid, _) in group.iterrows():
        # print(qid, pid)
        qrels[idx].append((qid, 0, pid, 1))

        text = quries_df[quries_df['qid']==qid]['text'].values[0]
        quries[idx].append((qid, text))

In [25]:
for key in qrels.keys():
    df_qrels = pd.DataFrame(qrels[key], columns=['qid', '0', 'pid', '1'])
    df_queries = pd.DataFrame(quries[key], columns=['qid', 'text'])
    df_queries.drop_duplicates(inplace=True)

    path_qrels = f'/home/requirements/data/MSMARCO_splited/qrels{key+1}.dev.tsv'
    path_queries = f'/home/requirements/data/MSMARCO_splited/queries{key+1}.dev.tsv'

    df_qrels.to_csv(path_qrels, sep='\t', index=False)
    delete_first_row_tsv(path_qrels)

    df_queries.to_csv(path_queries, sep='\t', index=False)
    delete_first_row_tsv(path_queries)

Query splitter - batch 51 to 52

In [4]:
pid_in_batch = [i for i in range(7130310, 7409929)]


qrels = []
quries = []

for _, (qid, _, pid, _) in qrels_df.iterrows():
    if pid in pid_in_batch:
        qrels.append((qid, 0, pid, 1))

        text = quries_df[quries_df['qid']==qid]['text'].values[0]
        quries.append((qid, text))


df_qrels = pd.DataFrame(qrels, columns=['qid', '0', 'pid', '1'])
df_queries = pd.DataFrame(quries, columns=['qid', 'text'])
df_queries.drop_duplicates(inplace=True)

path_qrels = f'/home/requirements/data/MSMARCO_51to52/qrels.dev.tsv'
path_queries = f'/home/requirements/data/MSMARCO_51to52/queries.dev.tsv'

df_qrels.to_csv(path_qrels, sep='\t', index=False)
delete_first_row_tsv(path_qrels)

df_queries.to_csv(path_queries, sep='\t', index=False)
delete_first_row_tsv(path_queries)